### Land Cover Classification for Landsat Collection 2 Level 2 Data

This jupyter notebook belongs to the Spring Semester 2024, GEO441 Remote Sensing Seminar Project "Time Series Analysis of Mining Activities and Related
Pollution in La Rinconada, Peru Using Satellite Imagery" and provides the python code to the methodology for landcover classification, which is described in the seminar projects report. 

**Remarks:** 
- All data can be found in the Microsoft Teams Group 'Team Peru - Mining'. Please download the folder called 'data.zip', unzip it and store it in the same directory as this notebook.
- If the notebook is not accessed trough GitHub directly, please also download the python file 'fast_glcm' and store it in the same directory as this notebook as well. The file stems from a GitHub repository (Taka, 2022) providing a fast version of GLCM calculations.
- This notebook uses some python modules that might be difficult to install consecutively, due to inconsistency between package dependencies. It is therefor recommended to use a conda environment and the 'environment_larinconada.yml' file (also in this GitHub repository) to install the necessary infrastructure to run the classification.

**Source**:
*Taka, I. (2022).* Fast gray-level co-occurrence matrix by numpy. Retrieved 21.05.2024, from https://github.com/tzm030329/GLCM

In [ ]:
import rasterio
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.transform import from_origin
from rasterio.features import geometry_mask
import matplotlib
from matplotlib import pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import fiona
from sklearn import (model_selection, metrics)
from sklearn.ensemble import RandomForestClassifier
from shapely.geometry import Point
from scipy.ndimage import generic_filter
from scipy.stats import mode
import fast_glcm

**General Functions**

In [ ]:
# Normalize bands into 0.0 - 1.0 scale
def normalize(array):
    array_min, array_max = array.min(), array.max()
    return (array - array_min) / (array_max - array_min)

# Create a RGB image
def get_rgb(raster):
    red = raster.read(3)
    green = raster.read(2)
    blue = raster.read(1)
    
    # Normalize band DN
    red_norm = normalize(red)
    green_norm = normalize(green)
    blue_norm = normalize(blue)
    
    # Stack bands
    nrg = np.dstack((red_norm, green_norm, blue_norm))

    #View the color composite
    plt.imshow(nrg)

# Create a False Color Composite
def get_fcc(raster):
    nir = raster.read(4)
    red = raster.read(3)
    green = raster.read(2)
    
    # Normalize band DN
    nir_norm = normalize(nir)
    red_norm = normalize(red)
    green_norm = normalize(green)
    
    # Stack bands
    nrg = np.dstack((nir_norm, red_norm, green_norm))
    
    # View the color composite
    plt.imshow(nrg)

**Loading datasets**

Please choose what sample dataset to use. 
- To classify images from 1999-2012, use the "Landsat7_groundcovers.shp".
- To classify images from 2013-2023, use the "Landsat8_groundcovers.shp".

In [ ]:
# load pixel center shapefile
point_grid_gdf = gpd.read_file(r"data/input_data/Landsat_30m_PointGrid.shp")

#Insert dataset path for either Landsat 7 or Landsat 8!
ground_samples_gdf = gpd.read_file(r"data/input_data/Landsat7_groundcovers.shp")

**Preprocessing of datasets**

In the following section the landcover samples are preprocessed and prepared for training the classifier.

In [ ]:
band_names = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2'
              ,'blue_glcm', 'green_glcm', 'red_glcm', 'nir_glcm', 'swir1_glcm', 'swir2_glcm'
              ,'ndvi','ndwi', 'smi']
band_indices = [1, 2, 3, 4, 5, 6, 7 , 8, 9, 10, 11, 12, 13, 14, 15]

Create Point Layer that contains only points within the our polygon samples and join the band values to the points. This creates a point layer that will later be split into test and training data.

In [ ]:
# get a list of all years represented in the ground samples, initialize a dataframe containing all necessary columns for an 
# initial version of the training and test dataframe.
ground_sample_years = list(np.unique(ground_samples_gdf.year))
sample_points = pd.DataFrame({
    'id': [],
    'geometry': [],
    'index_right': [],
    'land_cover': [],
    'year': [],
    'blue': [],
    'green': [],
    'red': [],
    'nir': [],
    'swir1': [],
    'swir2': [],
    'blue_glcm':[],
    'green_glcm':[],
    'red_glcm':[],
    'nir_glcm':[],
    'swir1_glcm':[],
    'swir2_glcm':[],
    'ndwi':[],
    'ndvi':[],
    'smi':[]
})

In the following cell, for each year...
**1)** the pixel centers that are contained within a sample polygon from that respective year are extracted,
**2)** the landsat image of this year is opened and the bands are normalized, indices and glcm entropy is calculated and all those bands are stored in a new raster image,
**3)** the new raster image is reopened, and intersected with the pixel centers extracted in step 1, then added to the collection of all sample points.

**4)** After those steps, the full sample_points dataframe is balanced by the smallest number of landcover classes contained in it.

In the subsequent cells the inputs will further be normalized and the coordinates added to the input dataset. Finally the test and training classes are created.

In [ ]:
for year in ground_sample_years:
    #step 1
    sample_point_year = gpd.sjoin(point_grid_gdf, ground_samples_gdf[ground_samples_gdf.year == year], how='inner', predicate='intersects')
    coord_list = [(x, y) for x, y in zip(sample_point_year["geometry"].x, sample_point_year["geometry"].y)]

    #step 2
    with rasterio.open(r"data/landsat_images/PE_LX-sr_30m_%s_composite_filt.tif"%str(year)) as src:
        blue = normalize(src.read(1).astype(float))
        green = normalize(src.read(2).astype(float))
        red = normalize(src.read(3).astype(float))
        nir = normalize(src.read(4).astype(float))
        swir1 = normalize(src.read(5).astype(float))
        swir2 = normalize(src.read(6).astype(float))

        ndwi = (green - nir) / (green + nir)
        ndvi = (nir - red) / (nir + red)
        smi = (nir - swir1) / (nir + swir1)
 
        # Get metadata of the original image
        profile = src.profile
        
        # Update metadata to include index and glcm entropy bands
        profile.update(
            count=src.count + 9,  # Increase band count by 9
            dtype=rasterio.float32  # Set data type of new band to float32
        )
        
        # Calculate glcm entropy for each original landsat band        
        glcm_means = []
        for i in range(6):
            img = src.read(i+1)
            h, w = img.shape
            mean_glcm = fast_glcm.fast_glcm_entropy(img) # Calculate pixel entropy
            glcm_means.append(mean_glcm)

        # After reading, close the raster file before opening it for writing
        image_path = r"data/landsat_images/PE_LX-sr_30m_%s_composite_filt_extended.tif" % str(year)
        
        with rasterio.open(image_path, 'w', **profile) as dst:
            # Write the original bands to the updated image
            for i in range(1, src.count + 1):
                dst.write(normalize(src.read(i)), i)
            # Write glcm bands to updated image    
            for i in range(6):
                dst.write(glcm_means[i], src.count+(i+1))
            # Write indices to updated image
            dst.write(ndvi, src.count + 7)
            dst.write(ndwi, src.count + 8)
            dst.write(smi, src.count + 9)
            
   #step 3
    with rasterio.open(image_path) as image:
        for i in band_indices: 
            sample_point_year[band_names[i-1]] = [x[0] for x in image.sample(coord_list, indexes=[i])]
        sample_points = pd.concat([sample_point_year, sample_points], ignore_index=True)

#step 4
# Get the class labels and count the number of samples per class
class_counts = sample_points['land_cover'].value_counts()

# Find the class with the lowest number of samples
min_class = class_counts.idxmin()
min_samples = class_counts[min_class]

# Create a balanced dataset
balanced_dataset = pd.DataFrame()

for class_label in class_counts.index:
    # Get the samples for the current class
    class_samples = sample_points[sample_points['land_cover'] == class_label]
    
    # If the class has more samples than the minimum, randomly select the minimum number of samples
    if len(class_samples) > min_samples:
        balanced_class_samples = class_samples.sample(n=min_samples, random_state=42)
    else:
        balanced_class_samples = class_samples
    
    # Append the balanced class samples to the balanced dataset
    balanced_dataset = pd.concat([balanced_dataset, balanced_class_samples], ignore_index=True)

# Shuffle the balanced dataset
sample_points = balanced_dataset.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
if 'index_right' in sample_points.columns:
        sample_points.drop(columns=['index_right'], inplace=True)

In [ ]:
#Normalize GLCM Bands between 0 and 1
glcm_bands=['blue_glcm', 'green_glcm', 'red_glcm', 'nir_glcm', 'swir1_glcm', 'swir2_glcm']
for band in glcm_bands:
    sample_points[band] = sample_points[band]/np.max(sample_points[band])

In [ ]:
# Analyse mean band values per land cover
means_per_land_cover = sample_points.groupby('land_cover')[band_names].mean()

# Plotting line plot
plt.figure(figsize=(12, 6))
for land_cover, data in means_per_land_cover.iterrows():
    plt.plot(data.index, data.values, label=land_cover)

plt.xlabel('Band')
plt.ylabel('Mean Values')
plt.title('Mean Values of Bands per Land Cover')
plt.legend(title='Land Cover')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Add pixel coordinates to the training and test datasample
training_bands = band_names.copy()
training_bands.append('X')
training_bands.append('Y')
# Normalize Coordinates between 0 and 1
sample_points['X'] = sample_points.geometry.x/np.max(sample_points.geometry.x)
sample_points['Y'] = sample_points.geometry.y/np.max(sample_points.geometry.y)

In [ ]:
# Create Training and Test Data Sets
X = sample_points[training_bands].fillna(0)
y = sample_points['land_cover']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.4, random_state=42)

**Model Creation**

In [ ]:
# Initialisation of Classifier
lc_classifier= RandomForestClassifier(random_state=100)

lc_classifier.fit(X_train, y_train)
score_training = lc_classifier.score(X_train,y_train)
score_test = lc_classifier.score(X_test,y_test)

# Print Overall Accuracy
print("Overall Accuracy: %.2f %%"%(score_test*100))
print("Training Accuracy: %.2f %%"%(score_training*100))

**Model Quality Metrics**

In [ ]:
## Confusion Matrix
from sklearn.metrics import confusion_matrix

classifier = lc_classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

# normalized confusion matrix
cm_norm = cm.astype('float') / cm.sum(axis=1)[:,np.newaxis]
np.set_printoptions(suppress=True, precision=3)
print(cm_norm)

In [ ]:
sample_points['land_cover'].unique()

In [ ]:
# visualize cm_norm
import seaborn as sns

colors = ['#FF0000', '#00FF00']  
cmap = sns.color_palette(colors)
labels = sample_points['land_cover'].unique()

plt.figure(figsize=(12,10))
figure = plt.gcf()
ax = figure.add_subplot(111)
sns.heatmap(cm_norm, cmap=cmap, annot=True, annot_kws={"size": 8}, xticklabels=labels, yticklabels=labels)  # fmt='g' stellt sicher, dass die Werte als Zahlen angezeigt werden
ax.set_xlabel('Prediction')
ax.set_ylabel('Ground Truth')
ax.set_title(f'Confusion Matrix for Random Forest Classifier')
plt.savefig('cm_rf_landsat7.png',  bbox_inches='tight', pad_inches=0.5)
plt.show()

In [ ]:
## Accuracy
from sklearn.metrics import accuracy_score
ac = accuracy_score(y_test, y_pred)
print(ac)

In [ ]:
## Precision and Recall
from sklearn.metrics import classification_report
y_true, y_pred, = y_test, classifier.predict(X_test)
report = classification_report(y_true, y_pred, digits=2, target_names=labels, output_dict=True)

report = pd.DataFrame(report).transpose()
report

**Landcover Prediction**

First choose the years to predict. Then, the images are prepared like the training data, except that the intersection 
with the training samples is omitted. With those pixel centers and their input values in a dataframe, the classification can be
carried out.

In [ ]:
# Choose years to predict
# Landsat 7 = list(range(1999, 2013))
# Landsat 8 = list(range(2013, 2024))
years = list(range(1999, 2013))

In [ ]:
# Initializing input classification datastructure
landcover_identification = {
    'land_cover': ['barren land', 'bare rock', 'vegetation', 'mining area (land)', 'mining area (water)', 'urban area', 'water',
       'snow', 'shadow'],
    'lc_id': [1, 2, 3, 4, 5, 6, 7, 8,9]
}
landcover_identification = pd.DataFrame(landcover_identification)

In [ ]:
# This function was initially used to smooth the classification with a 3x3 majority filter. 
# The smoothened version was never used for any analysis, but still is in the output images as an eight band.
def majority_filter(neighborhood):
    return mode(neighborhood, axis=None)[0]

In [ ]:
for year in years:
    if (year not in ground_sample_years):
        with rasterio.open(r"data/landsat_images/PE_LX-sr_30m_%s_composite_filt.tif"%str(year)) as src:
            blue = normalize(src.read(1).astype(float))
            green = normalize(src.read(2).astype(float))
            red = normalize(src.read(3).astype(float))
            nir = normalize(src.read(4).astype(float))
            swir1 = normalize(src.read(5).astype(float))
            swir2 = normalize(src.read(6).astype(float))
            ndwi = (green - nir) / (green + nir)
            ndvi = (nir - red) / (nir + red)
            smi = (nir - swir1) / (nir + swir1) 
    
            # Get metadata of the original image
            profile = src.profile
            profile.update(
                count=src.count + 9,  # Increase band count by 9
                dtype=rasterio.float32  # Set data type of new band to float32
            )
            
            glcm_means = []
            for i in range(6):
                img = src.read(i+1)
                h, w = img.shape
                mean_glcm = fast_glcm.fast_glcm_entropy(img)
                glcm_means.append(mean_glcm)

    
        # After reading, close the raster file before opening it for writing
            image_path = r"data/landsat_images/PE_LX-sr_30m_%s_composite_filt_extended.tif" % str(year)
            
            with rasterio.open(image_path, 'w', **profile) as dst:
                # Write the original bands to the updated image
                for i in range(1, src.count + 1):
                    dst.write(normalize(src.read(i)), i)

                for i in range(6):
                    dst.write(glcm_means[i], src.count+(i+1))
                # Write indices
                dst.write(ndvi, src.count + 7)
                dst.write(ndwi, src.count + 8)
                dst.write(smi, src.count + 9)
                    
    # Sample pixel centers over newly saved image to retrieve band values per pixel in a dataframe-like structure
    with rasterio.open(r"data/landsat_images/PE_LX-sr_30m_%s_composite_filt_extended.tif"%str(year)) as dataset_to_classify:
        input_points = point_grid_gdf
        
        coord_list = [(x, y) for x, y in zip(input_points["geometry"].x, input_points["geometry"].y)]
        
        for i in band_indices: 
            input_points[band_names[i-1]] = [x[0] for x in dataset_to_classify.sample(coord_list, indexes=[i])]

    if 'index_right' in input_points.columns:
            input_points.drop(columns=['index_right'], inplace=True)
    
    # Remove all empty rows
    non_zero_rows = input_points.loc[~(input_points[band_names] == 0).all(axis=1)]
    input_points_c = non_zero_rows

    input_points_c['X'] = input_points_c.geometry.x/np.max(input_points_c.geometry.x)
    input_points_c['Y'] = input_points_c.geometry.y/np.max(input_points_c.geometry.y)
    
    # Predict values, then merge coordinates, bands and classification back together
    X_to_predict = input_points_c[training_bands]
    X_to_predict = X_to_predict.fillna(0)
    y_pred = lc_classifier.predict(X_to_predict)
    output_points = input_points_c
    output_points['land_cover'] = y_pred
   
    # Join landcover identification with the output points
    mask = (output_points[['blue', 'green', 'red', 'nir', 'swir1', 'swir2']] == 0).all(axis=1)
    output_points.loc[mask, 'land_cover'] = "shadow"
    output_points = output_points.merge(landcover_identification, on='land_cover')
    
    # Get Metadata of initial raster image
    with rasterio.open(r"data/landsat_images/PE_LX-sr_30m_%s_composite_filt.tif"%str(year)) as src:
        # Read the raster dataset as a NumPy array
        raster_array = src.read()
        # Get the metadata of the raster dataset
        meta = src.meta
    
    # Extract coordinates and attribute values from points
    point_coords = [(x, y) for x, y in zip(output_points.geometry.x, output_points.geometry.y)]
    attribute_values = output_points['lc_id'].tolist()
    
    # Convert point coordinates to pixel coordinates
    pixel_coords = [src.index(x, y) for x, y in point_coords]
    
    # Create new layer in raster dataset 
    lc_id = np.empty((1,raster_array.shape[1], raster_array.shape[2]))
    raster_array = np.append(raster_array, lc_id, axis = 0)
    
    # Assign attribute values to corresponding raster pixels
    for (col, row), value in zip(pixel_coords, attribute_values):
        raster_array[6, col, row] = value
    
    # Create a new layer in the raster dataset
    new_layer_name = 'lc_id'
    meta['count'] += 1  # Increment the number of bands
    meta['dtype'] = raster_array.dtype  # Update the data type
    # Add smoothened version of landcover classification
    meta['count'] += 1 
    lc_smooth = np.empty((1,raster_array.shape[1], raster_array.shape[2]))
    raster_array = np.append(raster_array, lc_smooth, axis = 0) 
    raster_array[7] = generic_filter(raster_array[6], majority_filter, size=3)
    
    
    # Write the modified raster dataset with the new layer added
    with rasterio.open(r"data/outputs/landsat_%s_LC_RF.tif"%str(year), 'w', **meta) as dst:
        dst.write(raster_array)
        
    print("New layer added to raster dataset from %s."%str(year))